In [110]:
import json
import os
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [111]:
#access data using API
conn = http.client.HTTPSConnection("api.covid19api.com")
pl = '' #to store data
hs = {} #to store algorithms
conn.request("GET", "/summary", pl, hs)
resp = conn.getresponse()
data = resp.read().decode('UTF-8')

In [112]:
#convert data to JSON
covid = json.loads(data)

In [113]:
#normalizing data
pd.json_normalize(covid['Countries'], sep=",");

In [114]:
df = pd.DataFrame(covid['Countries'])
df.head();

In [115]:
covid19 = df.drop(columns=['CountryCode', 'Slug', 'Date', 'Premium'], axis=1)
covid19;

In [116]:
#creating base map
bm = folium.Map(tiles="Stamen Terrain", zoom_start=4)
bm;

In [117]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [118]:
covid19.dtypes;

In [119]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 4,
    name = 'Covid-19 Stats',
    data = covid19,
    columns = ['Country', 'TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nan_fill_color = 'black',
    legend_name = 'Total Confirmed Cases of Covid-19',
).add_to(bm)

bm;

In [120]:
covid19.update(covid19['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid19.update(covid19['TotalRecovered'].map('Total Recovered:{}'.format))

In [121]:
# coordinates of all countries in the world
coord = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coord;

In [122]:
covid19_final = pd.merge(covid19, coord, on='Country')
covid19_final;

In [123]:
def plot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius = 5,
                        weight = 2,
                        popup = [point.Country, point.TotalConfirmed, point.TotalRecovered],
                        fill_color = '#000000').add_to(bm)

<h2>Total Confirmed Cases of Covid-19 Worldwide grouped by Country</h2>

In [124]:
covid19_final.apply(plot, axis=1)
bm.fit_bounds(bm.get_bounds())
bm

In [125]:
#heat map
hm = folium.Map(tiles="StamenToner", min_zoom = 2)
hm;

In [126]:
death = covid19_final['TotalDeaths'].astype('float')
lat = covid19_final['latitude'].astype('float')
long = covid19_final['longitude'].astype('float')

In [127]:
hm.add_child(HeatMap(zip(lat,long,death,), radius=0));

In [128]:
def plot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                       radius = 5,
                       weight = 2,
                       popup = [point.Country, point.TotalDeaths],
                       fill_color = '#000000').add_to(hm)

<h2>Heat Map - Total Deaths due to Covid-19 Worldwide grouped by Country</h2>

In [129]:
covid19_final.apply(plot, axis=1)
hm.fit_bounds(hm.get_bounds())
hm